In [1]:
%cd ../..
# %pip install .
import time
from os import environ
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50, 
#         with viewAngle 135 deg
# ped = Pedestrian on roadsOrIntersections,
#         with regionContainedIn roadRegion,
#         facing Range(-180, 180) deg

# require abs(relative heading of ped from ego) > 70 deg

/home/yongming/workspace/research/apperception_new_local/apperception


In [2]:
environ['AP_PORT'] = str(input('port')) # README command uses port=25432

port25432


In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'pedestrian'
) as pedestrians

select *
from pedestrians as t1
join Cameras as cam on t1.cameraId = Cameras.id
where
  t1.heading < 180 and t1.heading > -180 and
  (contained(t1.traj, road_segment("roads"), cam.timestamp) or
   contained(t1.traj, road_segment("Intersections"), cam.timestamp)) and
  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -70 OR
  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > 70) AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation, Cameras.egoHeading), cam.timestamp) >= -15 AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation, Cameras.egoHeading), cam.timestamp) <= 15 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
  viewAngle(t1.traj, cam.egoHeading, cam.ego_translation, cam.timestamp) < 135
"""

In [3]:
### Prepare the world for queries ###
from apperception.world import empty_world
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.database import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F
# world = world.filter(lambda obj: obj.object_type == 'human.pedestrian.adult' or
#                                  obj.object_type == 'human.pedestrian.wheelchair' or
#                                  obj.object_type == 'human.pedestrian.personal_mobility' or
#                                  obj.object_type == 'human.pedestrian.construction_worker' or
#                                  obj.object_type == 'human.pedestrian.police_officer' or
#                                  obj.object_type == 'human.pedestrian.child' or
#                                  obj.object_type == 'human.pedestrian.stroller')
def pred(obj1, cam):
    return (
        obj1.heading < 180 and obj1.heading > -180 and
        F.contained(cam.ego, F.road_segment("road"), cam.timestamp) and
        (F.contained(obj1.traj, F.road_segment("road"), cam.timestamp) or
            F.contained(obj1.traj, F.road_segment("intersection"))) and
        (F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or
            F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and
        F.viewAngle(obj1, cam.ego, cam.timestamp) < 135
    )

# With road direction
world = world.filter("lambda obj1, cam: " +
        "F.like(obj1.object_type, 'human.pedestrian%') and " +
        "F.contained(cam.ego, F.road_segment('road')) and " +
        "(F.contained(obj1.traj, F.road_segment('lane'), cam.timestamp) or " + 
            "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp)) and " +
        "(F.facing_relative(obj1, cam.ego, cam.timestamp) < -70 or " + 
            "F.facing_relative(obj1, cam.ego, cam.timestamp) >= 70) and " + 
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) >= -15 and " +
        "F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp, cam.ego), cam.timestamp) <= 15 and " +
        "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
        "F.view_angle(obj1, cam.ego, cam.timestamp) < 67.5")

start = time.time()
# keys = world.get_traj_key()
id_time_camId_filename = world.get_id_time_camId_filename(num_joined_tables=1)

end = time.time()
print(format(end-start))

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT table_0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as table_0
        
        JOIN Cameras USING (cameraId)
        WHERE ((table_0.objectType LIKE 'human.pedestrian%') AND contained(Cameras.egoTranslation, roadSegment('road')) AND (contained(table_0.trajCentroids, roadSegment('lane'), Cameras.timestamp) OR contained(table_0.trajCentroids, roadSegment('intersection'), Cameras.timestamp)) AND ((facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp)<(-70)) OR (facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp)>=70)) AND (facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp)>=(-15)) AND (facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cam

In [4]:
resultImages = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename:
    itemId, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    resultImages[filename] = (itemId, timestamp, camId)

In [5]:
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import os
import random

experiment_data_dir =  "data/nuscenes/experiment_data"
# experiment_data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data'
def show_images(images, sample=None):
    if sample is not None:
        images = [i for i in images]
        random.shuffle(images)
        images = images[:sample]
    
    plt.figure(figsize=(60,30))
    columns = 3

    for i, image in enumerate(images):
        print("image", image)
        img = mpimg.imread(os.path.join(experiment_data_dir, image))
        print("loaded")
        plt.subplot(len(images) // columns + 1, columns, i + 1)
        plt.imshow(img)

In [6]:
show_images(list(resultImages.keys()), sample=10)

image n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148862404.jpg


FileNotFoundError: [Errno 2] No such file or directory: 'data/nuscenes/experiment_data/n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148862404.jpg'

<Figure size 4320x2160 with 0 Axes>

In [ ]:
expected_imgs = ['n008-2018-05-21-11-06-59-0400__CAM_FRONT__1526915471412465.jpg', 'n008-2018-07-27-12-07-38-0400__CAM_FRONT__1532707917112404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153162404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385153662404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385154162404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385154662404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385155162404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385158662404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385159162404.jpg', 'n008-2018-08-27-11-48-51-0400__CAM_FRONT__1535385159662404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656805162404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656805662415.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656806162404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656806612404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656807162404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656807662404.jpg', 'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535656808162404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659404362404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659404762404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659405262404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659405762404.jpg', 'n008-2018-08-30-15-52-26-0400__CAM_FRONT__1535659406262404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535728830362404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729326412404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729326912404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729327412404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729327912404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328412404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729328912404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729329412404.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729329912795.jpg', 'n008-2018-08-31-11-19-57-0400__CAM_FRONT__1535729330362404.jpg', 'n008-2018-08-31-11-37-23-0400__CAM_FRONT__1535730293412404.jpg', 'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731236162404.jpg', 'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731236662404.jpg', 'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731237112404.jpg', 'n008-2018-08-31-11-56-46-0400__CAM_FRONT__1535731237612404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293291162404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293291662404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293292162404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293292662404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293293162404.jpg', 'n008-2018-09-18-13-41-50-0400__CAM_FRONT__1537293293662404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299143862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299144362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299144862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299145362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299145862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299146362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299146862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299147362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299147862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299148862404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299149412404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299229362404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299239112404.jpg', 'n008-2018-09-18-15-26-58-0400__CAM_FRONT__1537299240112404.jpg']

In [ ]:
show_images(expected_imgs, sample=10)

In [ ]:
missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in resultImages.keys():
        missing += 1
        missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
show_images(missing_images, sample=10)

In [ ]:
extra = 0
extra_images = {}
for img in resultImages.keys():
    if img not in expected_imgs:
        extra += 1
        extra_images[img] = resultImages[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(resultImages.keys()), "=", 100 * extra / len(resultImages.keys()), "%")
show_images(extra_images.keys(), sample=10)

In [ ]:
######## Getting info about tthe extra images ########

In [ ]:
from apperception.database import database
for img in extra_images:
    itemId, timestamp, camId = extra_images[img]
    prefix = "samples/CAM_FRONT/"
    query = f"SELECT \'Cameras: \', Cameras.filename, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.egoHeading, ST_X(Cameras.egoTranslation), ST_Y(Cameras.egoTranslation), ST_Z(Cameras.egoTranslation), \'Pedestrian: \', getX(table_0.trajCentroids, Cameras.timestamp), getY(table_0.trajCentroids, Cameras.timestamp), ST_Z(valueAtTimestamp(table_0.trajCentroids, Cameras.timestamp)), valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp) FROM Item_General_Trajectory AS table_0, Cameras" + \
                f" WHERE Cameras.filename = \'{prefix + img}\' AND table_0.itemId = \'{itemId}\'"
    database.cursor.execute(query)
    result = database.cursor.fetchall()
    print(result)

In [ ]:
######## Seeing what predicates are not being satisfied for missing images ########

In [ ]:
# We assume for now that there is no problem with the commented out predicates, since they are relativley simple
predicates = [
    # "(table_0.objectType LIKE 'human.pedestrian%')",
    "contained(Cameras.egoTranslation, roadSegment('road'))",
    "(contained(table_0.trajCentroids, roadSegment('road'), Cameras.timestamp) OR contained(table_0.trajCentroids, roadSegment('intersection'), Cameras.timestamp))",
    "((facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp)<(-70)) OR (facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp)>=70))",
    "(facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp)>=(-15))",
    "(facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp)<=15)",
    # "(distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50)",
    # "(viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<135)"
]

In [ ]:
from apperception.database import database
preds_missed = {}
for img in missing_images:    
    prefix = "samples/CAM_FRONT/"
    query = f"SELECT COUNT(*) FROM Item_General_Trajectory AS table_0, Cameras" + \
                f" WHERE Cameras.filename = \'{prefix + img}\' AND (table_0.objectType LIKE 'human.pedestrian%') AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<135)" 
    database.cursor.execute(query)
    result = database.cursor.fetchall()
    num_peds = result[0][0]
    # For now we only take a look at images with one pedestrian (so that we know that that one pedestrian is the one that is supposed to satisfy the condition)
    # (if still experiencing problems after fixing the errors with the images with only one pedestrian, we can expand)
    if num_peds != -1:
        # print("------", img, "------")
        for predicate in predicates:
            prefix = "samples/CAM_FRONT/"
            query = f"SELECT true FROM Item_General_Trajectory AS table_0, Cameras" + \
                        f" WHERE Cameras.filename = \'{prefix + img}\' AND (table_0.objectType LIKE 'human.pedestrian%') AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<135) AND " + \
                        predicate 
            database.cursor.execute(query)
            result = database.cursor.fetchall()
            print(result)
            # if the predictae was not satisfied
            if len(result) == 0: 
                if predicate not in preds_missed:
                    preds_missed[predicate] = 0
                preds_missed[predicate] += 1
                print(predicate)
                query = f"SELECT {predicate}, facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp), roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), ST_X(Cameras.egoTranslation), ST_Y(Cameras.egoTranslation), ST_Z(Cameras.egoTranslation), Cameras.egoHeading FROM Item_General_Trajectory AS table_0, Cameras" + \
                        f" WHERE Cameras.filename = \'{prefix + img}\' AND (table_0.objectType LIKE 'human.pedestrian%') AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<135)" 
                database.cursor.execute(query)
                result = database.cursor.fetchall()
                print(result)

            

In [ ]:
for pred in preds_missed:
    print(pred, preds_missed[pred])